# MCP Services Creation

In [ ]:
import os
import json

import pandas as pd
import numpy as np


from pprint import pprint
from langchain_openai import  AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv
from sqlalchemy import (
    create_engine, Column, Integer, String, Float, Date, Boolean, Text, JSON
)
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.schema import DDL, CheckConstraint
from tools.prompt import generate_few_shot_ner_prompts
from tools.schema import (
    NERDecisao,
    Obrigacao
)

from datetime import timedelta


load_dotenv()

gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)

'''
gpt4turbo = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.0
)
'''

extractor_gpt4turbo = gpt4turbo.with_structured_output(NERDecisao, include_raw=False, method="json_schema")
extractor_obrigacao_gpt4turbo = gpt4turbo.with_structured_output(
    Obrigacao, include_raw=False, method="json_schema")


def safe_int(value):
    if pd.isna(value):
        return None
    return int(value)

c:\Users\05911205424\Documents\Dev\decicontas.br\.venv\Lib\site-packages\langchain_openai\chat_models\base.py:1784: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [2]:
import pymssql
from sqlalchemy.engine import Engine
def get_connection(db: str = 'processo') -> Engine:
    load_dotenv()
    
    server = os.getenv("SQL_SERVER_HOST")
    user = os.getenv("SQL_SERVER_USER")
    password = os.getenv("SQL_SERVER_PASS")
    port = os.getenv("SQL_SERVER_PORT", "1433")  # default MSSQL port
    database = db

    # Construct connection string for SQLAlchemy using pymssql
    connection_string = f"mssql+pymssql://{user}:{password}@{server}/{database}"

    # Create and return SQLAlchemy engine
    engine = create_engine(connection_string)
    return engine


In [3]:
sql_unidades = """
SELECT [IdUnidadeJurisdicionada]
      ,[NomeUnidade]
  FROM [BdSIAI].[dbo].[Anexo42_UnidadeJurisdicionada]
"""
df_unidades = pd.read_sql(sql_unidades, get_connection('BdSIAI'))

In [4]:
sql_citacoes = """
SELECT cit.*
  FROM [processo].[dbo].[Cit_Citacoes] AS cit
  WHERE CONCAT(cit.Numero_Processo, '/', cit.Ano_Processo) = '{}'
"""

In [5]:
sql_obrigacoes = """
SELECT o.*, p.Numero_Processo, p.Ano_Processo, CONCAT(p.Numero_Processo, '/', p.Ano_Processo) AS Processo
FROM BdDIP.dbo.Obrigacao o
INNER JOIN processo.dbo.Processos p
ON o.IdProcesso = p.IdProcesso 
"""
df_obrigacoes = pd.read_sql(sql_obrigacoes, get_connection('BdDIP'))

In [60]:


def get_responsible_unit(id_unit: int) -> pd.DataFrame:
    sql_resp = """
    SELECT DISTINCT resp.Nome,
        uni.IdUnidadeJurisdicionada,
        uni.NomeUnidade,
        respuni.Cargo,
        respuni.DataInclusao,
        respuni.DataInicioGestao,
        respuni.DataTerminoGestao
    FROM [BdSIAI].[dbo].[Anexo42_Responsavel] resp
    INNER JOIN [BdSIAI].[dbo].[Anexo42_ResponsavelUnidade] respuni ON resp.IdResponsavel = respuni.IdResponsavel
    INNER JOIN [BdSIAI].[dbo].[Anexo42_UnidadeJurisdicionada] uni ON uni.IdUnidadeJurisdicionada = respuni.IdUnidadeJurisdicionada 
        WHERE uni.IdUnidadeJurisdicionada = {}
    """

    return pd.read_sql(sql_resp.format(id_unit), get_connection('BdSIAI'))


In [51]:
import unicodedata
from rapidfuzz import process, fuzz  # pip install rapidfuzz

def normalizar_texto(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    s = s.strip().upper()
    # Remove acentos
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return s

def encontrar_unidade(query: str, df_unidades, limite=5, score_cutoff=70):
    """
    Retorna as melhores correspondências de NomeUnidade para a string `query`.
    """
    # Garante cópia para não alterar o original
    df = df_unidades.copy()
    
    # Coluna normalizada (pode salvar de vez no DF se quiser)
    df["NomeUnidade_norm"] = df["NomeUnidade"].apply(normalizar_texto)
    query_norm = normalizar_texto(query)

    # Monta lista de opções a comparar
    nomes_norm = df["NomeUnidade_norm"].tolist()

    # Busca fuzzy
    resultados = process.extract(
        query_norm,
        nomes_norm,
        scorer=fuzz.WRatio,
        limit=limite,
        score_cutoff=score_cutoff,
    )

    # `resultados` é uma lista de tuplas: (string_match, score, índice)
    indices = [idx for _, _, idx in resultados]

    # Retorna subset com score
    df_result = df.iloc[indices].copy()
    df_result["score"] = [score for _, score, _ in resultados]
    # Ordena do melhor para o pior
    df_result = df_result.sort_values("score", ascending=False)

    # Só interessa as colunas originais + score
    return df_result[["IdUnidadeJurisdicionada", "NomeUnidade", "score"]]


In [55]:
matches1 = encontrar_unidade(
    "instituto de previdência social do rio grande do norte",
    df_unidades,
)
print(matches1.iloc[0]['IdUnidadeJurisdicionada'], matches1.iloc[0]['NomeUnidade'], matches1.iloc[0]['score'])

19 INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO ESTADO DO RIO GRANDE DO NORTE 88.0


In [19]:
import re
import unicodedata

# slugify helper
def slugify(name: str) -> str:
    if pd.isna(name):
        return ""
    s = str(name)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower()
    s = re.sub(r"[^\w\s-]", "", s)        # remove punctuation
    s = re.sub(r"[\s_-]+", "_", s)        # spaces and dashes -> underscore
    s = s.strip("_")
    return s or "col"

# build new unique column names
old_cols = list(df_obrigacoes.columns)
slugged = [slugify(c) for c in old_cols]

seen = {}
unique_cols = []
for name in slugged:
    count = seen.get(name, 0)
    if count:
        new_name = f"{name}_{count}"
    else:
        new_name = name
    seen[name] = count + 1
    unique_cols.append(new_name)

# apply to dataframe and show result
col_mapping = dict(zip(old_cols, unique_cols))
df_obrigacoes.rename(columns=col_mapping, inplace=True)

# quick check
print(df_obrigacoes.columns.tolist())
df_obrigacoes.head(3)

['idobrigacao', 'idprocesso', 'idcomposicaopauta', 'idvotopauta', 'descricaoobrigacao', 'defazer', 'prazo', 'datacumprimento', 'orgaoresponsavel', 'idorgaoresponsavel', 'temmultacominatoria', 'nomeresponsavelmultacominatoria', 'documentoresponsavelmultacominatoria', 'idpessoamultacominatoria', 'valormultacominatoria', 'periodomultacominatoria', 'emultacominatoriasolidaria', 'solidariosmultacominatoria', 'cancelado', 'numero_processo', 'ano_processo', 'processo']


,idobrigacao,idprocesso,idcomposicaopauta,idvotopauta,descricaoobrigacao,defazer,prazo,datacumprimento,orgaoresponsavel,idorgaoresponsavel,...,documentoresponsavelmultacominatoria,idpessoamultacominatoria,valormultacominatoria,periodomultacominatoria,emultacominatoriasolidaria,solidariosmultacominatoria,cancelado,numero_processo,ano_processo,processo
0,870,50964,116424,40909,"a Secretaria de Estado da Agricultura, da Pecu...",True,prazos razoáveis e previamente estabelecidos,None,"SECRETARIA DA AGRICULTURA, DA PECUÁRIA E DA PESCA",NaN,...,None,NaN,NaN,None,False,null,None,003507,2006,003507/2006
1,897,19456,117592,42069,"Após o trânsito em julgado deste decisum, reme...",True,None,None,Diretoria de Atos e Execuções (DAE),NaN,...,None,NaN,NaN,None,False,null,None,001372,2002,001372/2002
2,900,45183,117629,42117,"Com a publicação desta Decisão, o processo dev...",True,None,None,Desconhecido,NaN,...,None,NaN,NaN,None,False,null,None,003124,1999,003124/1999


In [20]:
df_obrigacoes

,idobrigacao,idprocesso,idcomposicaopauta,idvotopauta,descricaoobrigacao,defazer,prazo,datacumprimento,orgaoresponsavel,idorgaoresponsavel,...,documentoresponsavelmultacominatoria,idpessoamultacominatoria,valormultacominatoria,periodomultacominatoria,emultacominatoriasolidaria,solidariosmultacominatoria,cancelado,numero_processo,ano_processo,processo
0,870,50964,116424,40909,"a Secretaria de Estado da Agricultura, da Pecu...",True,prazos razoáveis e previamente estabelecidos,None,"SECRETARIA DA AGRICULTURA, DA PECUÁRIA E DA PESCA",NaN,...,None,NaN,NaN,None,False,null,None,003507,2006,003507/2006
1,897,19456,117592,42069,"Após o trânsito em julgado deste decisum, reme...",True,None,None,Diretoria de Atos e Execuções (DAE),NaN,...,None,NaN,NaN,None,False,null,None,001372,2002,001372/2002
2,900,45183,117629,42117,"Com a publicação desta Decisão, o processo dev...",True,None,None,Desconhecido,NaN,...,None,NaN,NaN,None,False,null,None,003124,1999,003124/1999
3,221,40404,107805,34682,NOTIFICAÇÃO do Instituto de Previdência respon...,True,None,None,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,536.0,...,None,NaN,NaN,None,False,null,None,002804,2007,002804/2007
4,490,73410,115301,39671,NOTIFICAÇÃO do Instituto de Previdência dos Se...,True,60 dias,2024-07-15,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,536.0,...,None,NaN,NaN,None,False,null,None,005194,2000,005194/2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,1652,611312,122255,45485,Determinar à Diretoria de Controle de Contas d...,True,None,None,"SECRETARIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, ...",513.0,...,None,NaN,NaN,None,False,null,None,301005,2025,301005/2025
1249,1744,615866,123214,45960,NOTIFICAÇÃO desta decisão ao Ministério Públic...,True,None,None,SECRETARIA MUNICIPAL DE SAÚDE DE NATAL,1276.0,...,None,NaN,NaN,None,False,null,None,002551,2025,002551/2025
1250,1745,615866,123214,45960,"Abertura da fase dialógica, por meio da citaçã...",True,prazo regimental,None,SECRETARIA MUNICIPAL DE SAÚDE DE NATAL,1276.0,...,None,NaN,NaN,None,False,null,None,002551,2025,002551/2025
1251,1746,615866,123214,45960,INTIMAÇÃO da presente decisão às partes envolv...,True,None,None,SECRETARIA MUNICIPAL DE SAÚDE DE NATAL,1276.0,...,None,NaN,NaN,None,False,null,None,002551,2025,002551/2025


In [32]:
obrigacao = df_obrigacoes[~df_obrigacoes['nomeresponsavelmultacominatoria'].isna()].iloc[0]
processo = obrigacao.processo

In [33]:
processo

'006846/2003'

In [34]:
df_citacoes = pd.read_sql(sql_citacoes.format(processo), get_connection('processo'))

In [35]:
df_citacoes.columns

Index(['Numero_Citacao', 'Ano_citacao', 'Sequencia', 'Numero_Processo',
       'Ano_Processo', 'Orgao', 'Nome', 'Logradouro', 'Complemento_Endereco',
       'Bairro', 'Cidade', 'UF', 'CEP', 'Tipo', 'Data_envio_AR',
       'Data_Inicio_Prazo_Resposta', 'Data_final_Prazo_resposta', 'Texto',
       'Data_chegada_resposta_DE', 'numero_processo_resposta',
       'ano_processo_resposta', 'Usuario_recebeu_resposta', 'Data_Resposta',
       'Procedimento', 'Observacao_procedimento', 'prazoProrrogacao',
       'numero_processo_solicitacao_prorrogacao',
       'ano_processo_solicitacao_prorrogacao', 'data_solicitacao_prorrogacao',
       'IdSessao', 'DataInclusao', 'UsuarioInclusao', 'InformacaoGerada',
       'IdCitacao', 'IdComunicacaoEletronica', 'CoditoTipoProcessoResposta',
       'IdPessoa', 'IdInformacao', 'NumeroPostagem', 'DataInicioContagem',
       'ProrrogacaoPrazo', 'ProrrogacaoPrazoDias', 'NovaDataInicial',
       'ComunicacaoPorEdital', 'TeveRecusa', 'DataPublicacaoDOE',
       'D

In [36]:
df_citacoes

,Numero_Citacao,Ano_citacao,Sequencia,Numero_Processo,Ano_Processo,Orgao,Nome,Logradouro,Complemento_Endereco,Bairro,...,IdInformacaoDeterminacao,Prazo,PrazoTipo,IdComunicacaoTipoEnvio,IdSetorInclusao,IdDiligenciaPai,IdCitacaoPai,IdTipoComunicacao,IsRecusaRecebimento,IdSessaoOperacao
0,000635,2019,1,006846,2003,SEARH,Pedro Dantas de Medeiros,Rua Kerginaldo Cavalcante nº 582,None,Nova Descoberta,...,None,None,None,None,None,None,None,None,None,None
1,001511,2024,1,006846,2003,SEARH,Instituto de Previdência Social do Rio Grande ...,Rua Jundiaí nº 410,None,Tirol,...,None,None,None,None,None,None,None,None,None,None
2,002898,2023,1,006846,2003,SEARH,Pedro Dantas de Medeiros,Rua Kerginaldo Cavalcante nº 582,None,Nova Descoberta,...,None,None,None,None,None,None,None,None,None,None
3,002899,2023,1,006846,2003,SEARH,Instituto de Previdência Social do Rio Grande ...,Rua Jundiaí nº 410,None,Tirol,...,None,None,None,None,None,None,None,None,None,None
4,002900,2023,1,006846,2003,SEARH,Ministerio Publico do Estado do Rio Grande do ...,"Rua Promotor Manoel Alves Pessoa Neto, 97",None,Candelaria,...,None,None,None,None,None,None,None,None,None,None
5,002901,2023,1,006846,2003,SEARH,RAIMUNDO BEVENUTO DA SILVA (Pedro Dantas de Me...,RUA RAIMUNDO CHAVES 1570,sala 206,CANDELÁRIA,...,None,None,None,None,None,None,None,None,None,None
6,002902,2023,1,006846,2003,SEARH,francisca margareth da silva coelho xavier (Pe...,RUA RAIMUNDO CHAVES 1570,sala 206,CANDELÁRIA,...,None,None,None,None,None,None,None,None,None,None
7,002917,2023,1,006846,2003,SEARH,supremo tribunal federal,"Praça dos três Poderes, no setor de Admt. Fede...",None,GABINETE MINISTRO DIAS TOFFOLI,...,None,None,None,None,None,None,None,None,None,None


In [47]:
obrigacao['orgaoresponsavel'].lower()

'instituto de previdência social do rio grande do norte'

In [41]:
obrigacao['nomeresponsavelmultacominatoria']

'SECRETARIA DE ESTADO DA ADMINISTRAÇÃO E RECURSO HUMANOS'

In [58]:
id_unidade = encontrar_unidade(obrigacao['orgaoresponsavel'], df_unidades).iloc[0]['IdUnidadeJurisdicionada']

In [61]:
get_responsible_unit(id_unidade)

,Nome,IdUnidadeJurisdicionada,NomeUnidade,Cargo,DataInclusao,DataInicioGestao,DataTerminoGestao
0,Demétrius Fernandes dos Santos,19,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,Coord. de Concessão de Benefícios Previdenciários,2018-12-17 08:10:00,2018-11-28,2018-12-31
1,GERALDO LAURENTINO DA SILVA JÚNIOR,19,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,ASSESSOR JURÍDICO 1ª CLASSE,2018-12-17 08:10:00,2015-01-29,2018-03-26
2,JOSÉ MARLÚCIO DIÓGENES PAIVA,19,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,PRESIDENTE,2018-12-17 08:10:00,2015-01-01,2018-12-31
3,LINAURA MARIA FREIRE SANTOS,19,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,Chefe de Gabinete,2020-03-11 14:16:00,2019-02-19,None
4,NEREU BATISTA LINHARES,19,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,PRESIDENTE,2019-01-09 13:25:00,2019-01-08,None
5,RICARDO LUIZ PEREIRA PINTO,19,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,ASSESSOR JURÍDICO,2018-04-03 10:12:00,2018-03-27,None
6,RICARDO LUIZ PEREIRA PINTO,19,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,Assessor Jurídico,2018-12-17 08:10:00,2018-03-27,2018-04-11
7,Tália Maia Lopes,19,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO EST...,Procuradora Geral,2018-12-17 08:10:00,2018-04-12,2018-11-06


In [43]:
obrigacao

idobrigacao                                                                           785
idprocesso                                                                          93668
idcomposicaopauta                                                                  112489
idvotopauta                                                                         36929
descricaoobrigacao                      Após cessar a aposentadoria da parte interessa...
defazer                                                                              True
prazo                                                                    30 dias corridos
datacumprimento                                                                2023-12-16
orgaoresponsavel                        Instituto de Previdência Social do Rio Grande ...
idorgaoresponsavel                                                                  512.0
temmultacominatoria                                                                  True
nomerespon

In [45]:
print(obrigacao['descricaoobrigacao'])

Após cessar a aposentadoria da parte interessada, desaverbar o período contributivo objeto da Certidão de Tempo de Contribuição emitida pelo INSS e devolvê-la à apontada Autarquia Federal, para que a parte interessada possa requerer, diretamente no INSS, sua aposentadoria, conforme as regras do RGPS, levando a Certidão de Tempo de Contribuição emitida pelo IPERN para averbação junto ao INSS.


In [50]:
df_unidades.columns

Index(['IdUnidadeJurisdicionada', 'NomeUnidade'], dtype='object')